In [3]:
import numpy as np
import pandas as pd
import nbimporter

from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import wordnet_ic, wordnet as wn
from sklearn.linear_model import LogisticRegression, Perceptron
from sklearn.tree import DecisionTreeClassifier
from nltk.metrics import *
from nltk.tag import *
from nltk.corpus import treebank
from sent_utils import *
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import Scatter, Figure, Layout
from nltk.stem.porter import PorterStemmer
init_notebook_mode(connected=True)
brown_ic =  wordnet_ic.ic('ic-brown.dat')
stemmer = PorterStemmer()

In [176]:
# Paraphrase detectors
def lemma_synset_jaccard(sent_0, sent_1):
    print('.', end='')
    sent_0, sent_1 = remove_punctuation(sent_0.lower()), remove_punctuation(sent_1.lower())
    words_0, words_1 = words_from_sent(sent_0), words_from_sent(sent_1)
    tokens_0, tokens_1 = tokens_from_words(words_0), tokens_from_words(words_1)
    lemmas_0, lemmas_1 = tokens_to_lemmas(tokens_0), tokens_to_lemmas(tokens_1)
    senses_0, senses_1 = tokens_to_senses(tokens_0), tokens_to_senses(tokens_1)
    return [
        jaccard_distance(set(lemmas_0), set(lemmas_1))
    ]

In [187]:
def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

def tokenize(text):
    tokens = word_tokenize(text)
    stems = stem_tokens(tokens, stemmer)
    return stems

flattened = []
for line in open('IHLT-eval-framework/train/msr_paraphrase_train_input.txt', encoding="UTF8").readlines():
    for s in line.split("\t"):
        flattened.append(s.strip())
    
tfidf = TfidfVectorizer(tokenizer=tokenize, stop_words='english')
tfs = tfidf.fit_transform(flattened)

In [188]:
input_sentences = text_to_sentences('IHLT-eval-framework/train/msr_paraphrase_train_input.txt')

In [165]:
X_train = [lemma_synset_jaccard(data[0], data[1]) for data in input_sentences]

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [166]:
y_train = open('IHLT-eval-framework/train/msr_paraphrase_train_gs.txt', encoding="UTF8").readlines()
y_train[0] = '1'
y_train = [int(line.strip()) for line in y_train]

In [167]:
test_input = text_to_sentences('IHLT-eval-framework/test/msr_paraphrase_test_input.txt')
X_test = [lemma_synset_jaccard(data[0], data[1]) for data in test_input]

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [189]:
y_test = open('IHLT-eval-framework/test/msr_paraphrase_test_gs.txt', encoding="UTF8").readlines()
y_test = [int(line.strip()) for line in y_test]

In [190]:
regression = LogisticRegression()
regression.fit(X_train, y_train)
prediction = regression.predict(X_test)
MSRP_eval(prediction, y_test)

acc= 0.7055072463768116  reject= 0.5988700564971752  accept= 0.7330415754923414


In [170]:
regression = DecisionTreeClassifier()
regression.fit(X_train, y_train)
prediction = regression.predict(X_test)
MSRP_eval(prediction, y_test)

acc= 0.648695652173913  reject= 0.47627118644067795  accept= 0.7383259911894273
